# StatLineTable plot voorbeeld

Hier geven we een voorbeeld hoe je makkelijk met een Python script data van statlijn kan halen en een plot in CBS stijl kan maken. 

We maken gebruik van de *StatLineTable* klasse uit de *cbs_utils.readers* sectie. Deze klasse is een toevoeging op de *cbsodata* module. Met *StatLineTable* kan je de data in een gestructureede pandas dataframe stoppen, zodat het bewerken en plotten een stuk eenvoudiger gaat. 

We beginnen met het inladen van de benodigde modules

In [1]:
import matplotlib.pyplot as plt
import logging

from cbs_utils.misc import (create_logger, merge_loggers)
from cbs_utils.plotting import CBSPlotSettings
from cbs_utils.readers import StatLineTable

Nu initialiseren we eerst de logger en ook de plot settings. Daar gebruiken we wat utilties uit cbs_utils voor

In [9]:
fig_properties = CBSPlotSettings()

logger = create_logger()
merge_loggers(logger, logger_name_to_merge="cbs_utils.readers")
logger2 = logging.getLogger("cbs_utils.readers")
logger2.setLevel(logging.INFO)

# Inlezen van een OpenData Tabel

Het inladen van een tabel uit opendata is eenvoudig. We moeten eerst de ID van een tabel vinden door naar statline te gaan en je tabel te openen. Bijvoorbeeld, het ICT gebruik van bedrijven in 2018 naar bedrijfsgrootte staat in deze tabel

https://opendata.cbs.nl/#/CBS/nl/dataset/84410NED/table?ts=1568706226304

Dit betekent dat de tabel ID '84410NED' is. Deze gaan we nu inlezen

In [10]:
%%time
table_id = "84410NED"
logger.info(f"Start met het lezen van tabel {table_id}")
statline = StatLineTable(table_id=table_id)

  INFO : Start met het lezen van tabel 84410NED
  INFO : Reading json cache/84410NED/DataProperties.json
  INFO : Reading json cache/84410NED/TypedDataSet.json
  INFO : Reading json cache/84410NED/TableInfos.json
  INFO : Reading from pickle database cache/84410NED_question.pkl
  INFO : Reading from pickle database cache/84410NED_section.pkl
  INFO : Reading from pickle database cache/84410NED_dimensions.pkl
  INFO : Writing table information to images/84410NED/TableInfos.yml
  INFO : Writing question structure to images/84410NED/QuestionTable.txt
  INFO : Writing question structure to images/84410NED/SectionTable.txt
CPU times: user 108 ms, sys: 8 ms, total: 116 ms
Wall time: 184 ms


Je ziet dat er 'under the hood' een hoop gebeurd. We zien de volgende stappen
    1. Het laden van de data wordt door de *cbsodata* module afgehandeld. Deze module slaat de data op in verschillende json file. 
    2. Vervolgens worden deze json file samengevoegd tot een pandas Dataframe. 
    3. We slaan de data frames op in een pickel file (standaar binary python file). De volgende keer als je dezelfde plot maakt zullen de pickel files ingelezen worden. Dit gaat veel sneller dan de de data nog een keer met *cbsodata* van het internet te downloadedn. Het maakt het ontwikkelen van een script een stuk gemakkelijker.
    4. We slaan de structuur van de tabel op in wat txt files: QuestionTable bevat alle vragen, SectionTable alle modules. Deze data files kan je gebruiken om snel op te zoeken welk nummer je moet gebruiken om een vraag te plotten
    
Om te laten zien dat het laden van de tabellen een tweede keer een stuk sneller gaat, draaien we de script nog een keer:

In [11]:
%%time
statline = StatLineTable(table_id=table_id)

  INFO : Reading json cache/84410NED/DataProperties.json
  INFO : Reading json cache/84410NED/TypedDataSet.json
  INFO : Reading json cache/84410NED/TableInfos.json
  INFO : Reading from pickle database cache/84410NED_question.pkl
  INFO : Reading from pickle database cache/84410NED_section.pkl
  INFO : Reading from pickle database cache/84410NED_dimensions.pkl
  INFO : Writing table information to images/84410NED/TableInfos.yml
  INFO : Writing question structure to images/84410NED/QuestionTable.txt
  INFO : Writing question structure to images/84410NED/SectionTable.txt
CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 178 ms


De eerste keer dat we de script draaide duurde het ruim 7 seconden, de tweede keer een fractie van een seconde. Dit tijdsvoordeel is vooral prettig als je de script vaak herdraaid om een plaatje te 'tunen'

## Analyse van de opendata tabel

Als volgende stap gaan we bekijken hoe de data structuur van de vragenlijst eruit ziet. Zoals gezegd, dit je ook door de inhoud van QuestionTable.txt in the image directory te bekijken. Maar we kunnen het ook direct opvragen:

In [12]:
statline.show_module_table()

  INFO : Structure of all modules
+------+------------+----------------------------------------+
|   ID |   ParentID | Title                                  |
|------+------------+----------------------------------------|
|    1 |            | Personeel en ICT                       |
|    4 |          1 | ICT-specialisten                       |
|    9 |          1 | ICT-beveiliging/bescherming data       |
|   13 |            | Toegang en gebruik internet            |
|   14 |         13 | Bedrijven met website                  |
|   16 |         14 | Website bevat                          |
|   23 |            | Cloud-diensten                         |
|   25 |         23 | Type cloud-diensten                    |
|   33 |         25 | Type server                            |
|   36 |            | Big-data-analyse                       |
|   38 |         36 | Bronnen big data voor analyse          |
|   43 |         36 | Wie analyseerde big data               |
|   46 |            |

We zien hier een lijst van modules en subsecties. Een module is een blok van vragen die bij één onderwerp horen. Ze worden in opendaten in één sectie ondergebracht. Een module heeft als eigenschap dat hij ook een ParentID heeft. De ParentID bewijst naar de voorgaande sectie waartoe de module behoort. Zo heeft module 14 met de titel 'Bedrijven met website' bijvoorbeeld als parent de sectie 13 'Toegang en gebruik internet'. Deze sectie 13 heeft geen parent, daarom weten we dat het geen module is, maar gewoon sectie indeling (je ziet dat de waarde ParentID die bij 13 hoort leeg is).  

we kunnen ook de volledige structuur van de vragenlijst bekijken. Dit doen we als volgt:

In [13]:
statline.show_question_table()

  INFO : Structure of all questions
+-------------------------------+-----------------------------------------+------------------------------------------+-----------------------------+
|                               | Key                                     | Title                                    | Unit                        |
|-------------------------------+-----------------------------------------+------------------------------------------+-----------------------------|
| (1, 2, nan, nan, nan)         | ICTPersAangenomenWillenAannemen_1       | ICT-pers. aangenomen/willen aannemen     | % van bedrijven             |
| (1, 3, nan, nan, nan)         | ICTVacaturesLastigTeVervullen_2         | ICT-vacatures lastig te vervullen        | % van bedrijven             |
| (1, 4, 5.0, nan, nan)         | ICTSpecialistenInLoondienst_3           | ICT-specialisten in loondienst           | % van werkzame personen     |
| (1, 4, 6.0, nan, nan)         | ICTSpecOntwEnOnderhoudSoftware_4    